# LU- и LUP-разложение

#### Дибель Софья Васильевна НПМбд-01-19

СЛАУ можно записать в матричном виде A * x = b. 
Если матрица A не вырождена, то решение можно найти в виде x = A^(-1) * b.

С другой стороны, можно искать решение методом LU и LUP разложений.

In [16]:
import numpy as np
import scipy.linalg

A = np.array([[0, 9, 3],
              [4, 1, 3],
              [7, 1, 10]],
             dtype=float)

b = np.array([1, 2, 3], dtype=float)

### LU-разложение

Представляем матрицу A в виде A = L * U, где L - нижняя треугольная матрица с единицами на диагонали, U - верхняя треугольная матрица.

In [18]:
def LU(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=float)
    
    for i in range(n):
        L[i + 1:, i] = U[i + 1:, i] / U[i, i]
        U[i + 1:] = (U[i + 1:] - L[i + 1:, i, np.newaxis] * U[i])
            
    return L, U

Проверяем: A = L * U

In [19]:
print('A:')
print(A)
print()

L, U = LU(A)

print('L * U')
print(L.dot(U))
print()

A:
[[ 0.  9.  3.]
 [ 4.  1.  3.]
 [ 7.  1. 10.]]

L * U
[[nan nan nan]
 [nan nan nan]
 [nan nan nan]]



<ipython-input-18-7a8a8a13b585>:7: RuntimeWarning: divide by zero encountered in true_divide
  L[i + 1:, i] = U[i + 1:, i] / U[i, i]
<ipython-input-18-7a8a8a13b585>:8: RuntimeWarning: invalid value encountered in multiply
  U[i + 1:] = (U[i + 1:] - L[i + 1:, i, np.newaxis] * U[i])
<ipython-input-18-7a8a8a13b585>:7: RuntimeWarning: invalid value encountered in true_divide
  L[i + 1:, i] = U[i + 1:, i] / U[i, i]


### LUP-разложение

В LU-разложении мы делим на диагональный элемент матрицы А, но может быть и так, что он равен нулю. Тогда мы можем в изначальной матрице переставлять строки, чтобы на диагонали был не 0, и запоминать перестановки в матрицу перестановок P.

In [20]:
def LUP(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    P = np.eye(n, dtype=np.double)

    for i in range(n - 1):
        if U[i, i] == 0:
            U[[i, i + 1]] = U[[i + 1, i]]
            P[[i, i + 1]] = P[[i + 1, i]]

    for i in range(n):
        L[i + 1:, i] = U[i + 1:, i] / U[i, i]
        U[i + 1:] = (U[i + 1:] - L[i + 1:, i, np.newaxis] * U[i])
    return (P, L, U)

Проверяем: A = P.T * L * U

In [21]:
P, L, U = LUP(A)

print('A')
print(A)
print()

print('P.T * L * U')
print(P.T.dot(L.dot(U)))
print()

A
[[ 0.  9.  3.]
 [ 4.  1.  3.]
 [ 7.  1. 10.]]

P.T * L * U
[[ 0.  9.  3.]
 [ 4.  1.  3.]
 [ 7.  1. 10.]]



Решаем систему L * y = b

In [22]:
def forward_substitution(L, b):
    n = A.shape[0]
    y = np.zeros_like(b, dtype=np.double)
    y[0] = b[0] / L[0, 0]
    
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
        
    return y

Решаем систему U * x = y

In [23]:
def back_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=np.double);

    x[-1] = y[-1] / U[-1, -1]

    for i in range(n - 2, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i:], x[i:])) / U[i, i]
        
    return x

### Солверы

In [24]:
def LU_solve(A, b):
    L, U = LU(A)
    y = forward_substitution(L, b)
    
    return back_substitution(U, y)

In [25]:
def LUP_solve(A, b):
    P, L, U = LUP(A)
    y = forward_substitution(L, np.dot(P.T, b))
    
    return back_substitution(U, y)

Решение через LU: он будет выдавать array([nan, nan, nan]), если на диагонали будет 0

In [26]:
LU_solve(A, b)

<ipython-input-18-7a8a8a13b585>:7: RuntimeWarning: divide by zero encountered in true_divide
  L[i + 1:, i] = U[i + 1:, i] / U[i, i]
<ipython-input-18-7a8a8a13b585>:8: RuntimeWarning: invalid value encountered in multiply
  U[i + 1:] = (U[i + 1:] - L[i + 1:, i, np.newaxis] * U[i])
<ipython-input-18-7a8a8a13b585>:7: RuntimeWarning: invalid value encountered in true_divide
  L[i + 1:, i] = U[i + 1:, i] / U[i, i]


array([nan, nan, nan])

Решение через LUP: подходит для матриц с нулем на диагонали

In [27]:
LUP_solve(A, b)

array([ 0.52777778,  0.13888889, -0.08333333])

Решение через numpy

In [28]:
np.linalg.solve(A, b)

array([ 0.52777778,  0.13888889, -0.08333333])

#### Вроде сходится

Основное преимущество этих методов заключается в том, что столбец b при решении СЛАУ используется только на заключительном этапе (в солверах). Таким образом, если решается серия СЛАУ с одной и той же матрицей коэффициентов A, но разными правыми частями b, то очень выгодно один раз вычислить LU-разложение матрицы A, а уже за тем решать конкретные системы, меняя столбец свободных членов.